<b><font size=4> Tune damped factor in DLT model with backtest engine


In this demo, we want to show how to use the utility `tune_damped_factor` to tune the hyper-parameter `damped_factor_fixed` in **orbit DLT** model. 

Note: only use this when you want a fixed damped facotr value. Otherwise, one can specify `damped_factor_min` and `damped_factor_max` to do the sampling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.dlt import DLT
from orbit.backtest.backtest import TimeSeriesSplitter, Backtest
from orbit.backtest.multibacktest import tune_damped_factor
from orbit.utils.metrics import mape, smape, wmape

%matplotlib inline
plt.style.use('ggplot')

## Load data

In [2]:
data_path = '../examples/data/iclaims_example.csv'
raw_data = pd.read_csv(data_path, parse_dates=['week'])
data = raw_data.copy()
print(data.shape)
data.head(5)

(443, 5)


,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


## Create a TimeSeriesSplitter

In [3]:
min_train_len = 380
forecast_len = 20
incremental_len = 20

splitter = TimeSeriesSplitter(
    data, min_train_len, incremental_len, forecast_len, 
    window_type='expanding', date_col='week')

## Create a DLT model

In [4]:
mod = DLT(
    date_col = 'week',
    response_col = 'claims',
    regressor_col = ['trend.unemploy', 'trend.filling', 'trend.job'],
    seasonality = 52,
    seed = 8888,
    predict_method = 'map',
    sample_method = 'map',
    is_multiplicative = True,
)

## Conduct a damped factor grid search

In [5]:
res = tune_damped_factor(model = mod, 
                         splitter = splitter,
                         damped_factor_grid = [0.8, 0.9, 0.99],
                         predicted_col = 'prediction',
                         metrics = {"smape": smape, "mape": mape})

Doing backtesting for damped factor 0.8
Doing backtesting for damped factor 0.9
Doing backtesting for damped factor 0.99


Now we sort the results in terms of OOT `smape` value. In terms of this, `dampted_factor_fixed = 0.2` is recommended.

In [6]:
res.sort_values('test_smape')

,damped factor,train_smape,train_mape,test_smape,test_mape
2,0.99,0.040752,0.040566,0.045307,0.044323
1,0.90,0.040723,0.040533,0.046178,0.045098
0,0.80,0.041619,0.041508,0.050071,0.048783
